In [1]:
import os

In [2]:
%pwd

'c:\\Users\\RaviB\\GitHub\\SleepEfficiencyML\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\RaviB\\GitHub\\SleepEfficiencyML'

In [5]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/RavinderRai/SleepEfficiencyML.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "RavinderRai"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "98ec3895bb5f99c79771a8d5e985b6c4917c40c7"

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [7]:
from sleep_efficiency.constants import *
from sleep_efficiency.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
    
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.XGBoost
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/RavinderRai/SleepEfficiencyML.mlflow"
        )

        return model_evaluation_config

In [21]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [22]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [23]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-06-27 10:21:47,071: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-27 10:21:47,074: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-27 10:21:47,078: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-06-27 10:21:47,080: INFO: common: created directory at artifacts]
[2024-06-27 10:21:47,081: INFO: common: created directory at artifacts/model_evaluation]
[2024-06-27 10:21:47,147: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)': /RavinderRai/SleepEfficiencyML.mlflow/api/2.0/mlflow/runs/create]
[2024-06-27 10:21:47,381: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2024/06/27 10:21:55 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 2
Created version '2' of model 'ElasticnetModel'.


## Testing Inference

In [9]:
from src.sleep_efficiency.pipeline.prediction import PredictionPipeline
import pandas as pd
import numpy as np

In [10]:
df_test = pd.read_csv('artifacts/data_transformation/test.csv')
df_test.head()

,Age,Gender,Bedtime,Wakeup time,Sleep duration,Sleep efficiency,Awakenings,Caffeine consumption,Alcohol consumption,Smoking status,Exercise frequency
0,31,1,0,9,8.5,0.86,0.0,1,0,1,0
1,40,1,22,4,6.0,0.52,2.0,0,1,1,1
2,57,1,1,9,8.0,0.76,3.0,0,1,0,1
3,32,0,0,8,8.0,0.90,1.0,0,0,0,0
4,50,1,0,7,7.0,0.71,3.0,0,1,0,0


In [11]:
X_test = df_test.drop(columns=['Sleep efficiency'])

In [18]:
data = X_test.iloc[3]
#data = np.array(data).reshape(1, 10)
list(data)

[32.0, 0.0, 0.0, 8.0, 8.0, 1.0, 0.0, 0.0, 0.0, 0.0]

In [19]:
list(X_test.columns)

['Age',
 'Gender',
 'Bedtime',
 'Wakeup time',
 'Sleep duration',
 'Awakenings',
 'Caffeine consumption',
 'Alcohol consumption',
 'Smoking status',
 'Exercise frequency']

In [24]:
pd.DataFrame([data], columns=list(X_test.columns))

,Age,Gender,Bedtime,Wakeup time,Sleep duration,Awakenings,Caffeine consumption,Alcohol consumption,Smoking status,Exercise frequency
3,32.0,0.0,0.0,8.0,8.0,1.0,0.0,0.0,0.0,0.0


In [27]:
obj = PredictionPipeline()
predict = obj.predict(pd.DataFrame([list(data)], columns=list(X_test.columns)))
predict

array([0.8569192], dtype=float32)